In [16]:
from bs4 import BeautifulSoup
from pandas.core.frame import DataFrame
import requests
import plotly.express as px
import pandas as pd
from plotly.offline import plot
import matplotlib.pyplot as plt
from requests.models import default_hooks

#podanie ilosći stron do przeszukania
Number_of_Pages=6

#podanie miast : pisane z małej litery
city = "lublin"


In [18]:
page = 1
data = []
while page != (Number_of_Pages + 1):
    url = f"https://www.otodom.pl/pl/oferty/sprzedaz/dom/"+ city +f"?page={page}"
    print(url)
    site = requests.get(url)
    soup = BeautifulSoup(site.content, 'html.parser')
    
    #Information about hauses
    description = [i.get_text().replace('Dom na sprzedaż: ', '').split(', ') for i in soup.select('p', class_= 'css-80g06k')]

    description =(description[:-1])
    
    data.append(description)
    print(len(description))
    page = page + 1

len(description)

#connect all data, from list
i = 1
all_data = data[0]

while i != Number_of_Pages:
    all_data = all_data+data[i]
    i = i + 1

https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=1
156
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=2
156
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=3
156
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=4
156
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=5
156
https://www.otodom.pl/pl/oferty/sprzedaz/dom/lublin?page=6
156


156

In [43]:
#Collect data
#Sample data from one advertisement:
# [['Lublin', 'lubelskie'], ['290\xa0000\xa0zł'], ['5 pokoi102 m²'], ['Biuro nieruchomości']]

#I)places-info about cities and district
places = all_data[::4]
#print("miejsca:",places)

#II)price
hous_price = all_data [1::4]
#print("cena",hous_price)

#III)rooms, area, msquar
rooms_area_msquar = all_data [2::4]
#print("ilosc pokoi i powierzchnia :",rooms_area_msquar)

#IV)developer name - not needed



In [44]:
#I)1)Cities

cities = [i[0] for i in places]
#print("cities:",cities)

In [21]:
#I)2)Districts
districts_tmp = [i[1:2] for i in places]
districts = [', '.join(i) for i in districts_tmp]
#print ("districts:",districts)

In [22]:
#II)Prices
price_tmp = [', '.join(i) for i in hous_price]
price = [float(i.replace('zł/mc', '').replace('\xa0', '').replace('zł', '')) for i in price_tmp ]
#print("price:",price)

In [24]:
#III)
house_tmp = [', '.join(i) for i in rooms_area_msquar]

numbers=[]
for i in house_tmp:
    newstr = ''.join((ch if ch in '0123456789.' else ' ') for ch in i)
    listOfNumbers = [float(i) for i in newstr.split()]
    numbers.append(listOfNumbers)

#1)Rooms
rooms =[i[0] for i in numbers]

#2) area
area = [i[-1] for i in numbers]



In [47]:
data = pd.DataFrame()
places = places
data['Miasto'] = cities
data['Dzielnica'] = districts
data['Cena'] = price
data['Metry'] = area
data['Pokoje'] = rooms

print(data)
type(data)


# determining the name of the file
file_name = 'Houses.xlsx'

# saving the excel
data.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

     Miasto        Dzielnica       Cena   Metry  Pokoje
0    Lublin         Sławinek   950000.0  194.00     5.0
1    Lublin     Konstantynów   800000.0  164.75     7.0
2    Lublin         Sławinek  2790000.0  400.00    10.0
3    Lublin  Majdan Tatarski   730000.0  152.50     3.0
4    Lublin         Sławinek  1570000.0  300.00     4.0
..      ...              ...        ...     ...     ...
323  Lublin         Szerokie  1200000.0  220.00     5.0
324  Lublin         Węglinek  2490000.0  550.00     7.0
325  Lublin        Dziesiąta  1150000.0  185.00     7.0
326  Lublin        lubelskie   879000.0  180.00     7.0
327  Lublin      Śródmieście   526500.0  117.00     4.0

[328 rows x 5 columns]
DataFrame is written to Excel File successfully.


In [48]:
print("Number of scrapped offers:",data['Miasto'].count())
print("The most frequently appearing district :",data['Dzielnica'].value_counts().idxmax())
print("The cheapest offer:",data['Cena'].min())
print("The most expensive offer:",data['Cena'].max())
print("Average house price",round(data['Cena'].mean(), 2))
print("Average price per meter", round(data['Cena'].sum() / data['Metry'].sum(), 2))
print("the average house size",round(data['Metry'].mean(), 2))
print("Average number of rooms", round(data['Pokoje'].mean(), 0))

Number of scrapped offers: 328
The most frequently appearing district : lubelskie
The cheapest offer: 250000.0
The most expensive offer: 9900000.0
Average house price 1142168.15
Average price per meter 5099.22
the average house size 223.99
Average number of rooms 7.0


In [49]:
df = pd.DataFrame(data['Dzielnica'].value_counts())

df.columns = ['Ilosc']
df.index.name = 'Dzielnica'
fig = px.pie(df, values="Ilosc", names=df.index, height=800)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [37]:
df = pd.DataFrame(data['Pokoje'].value_counts())

df.columns = ['Ilość ofert']
df.index.name = "Ilość pokoi w domu"
fig = px.bar(df, x=df.index, y='Ilość ofert', text='Ilość ofert')
fig.update_xaxes(range=[0,12])

fig.update_traces(marker_color='lightblue')
fig.update_layout(showlegend=False)
fig.show()



In [38]:

D = dict({
    '0-80': 0,
    '80-120': 0,
    '120-160': 0,
    '160-200': 0,
    '200-240': 0,
    '240-280': 0,
    '280+': 0,
})
for i in data['Metry']:
    if i >= 0 and i < 80:
        D['0-80'] += 1
    elif i >= 80 and i < 120:
        D['80-120'] += 1
    elif i >= 120 and i < 160:
        D['120-160'] += 1
    elif i >= 160 and i < 200:
        D['160-200'] += 1
    elif i >= 200 and i < 240:
        D['200-240'] += 1
    elif i >= 240 and i < 280:
        D['240-280'] += 1
    elif i >= 280:
        D['280+'] += 1

df = pd.DataFrame.from_dict(D, orient='index')
df.index.name = 'Metraż domu w m²'
df.columns=['Ilość ofert']
fig = px.line(df, x=df.index, y='Ilość ofert', text='Ilość ofert')
fig.update_traces(textposition='top right')
fig.update_layout(showlegend=False)

fig.show()



In [39]:

D = dict({
    '0-200000': 0,
    '200000-400000': 0,
    '400000-600000': 0,
    '600000-800000': 0,
    '800000-1000000': 0,
    '1000000-1200000': 0,
    '1200000-1400000': 0,
    '1400000-1600000': 0,
    '1600000+': 0,
})
for i in data['Cena']:
    if i >= 0 and i < 200000:
        D['0-200000'] += 1
    elif i >= 200000 and i < 400000:
        D['200000-400000'] += 1
    elif i >= 400000 and i < 600000:
        D['400000-600000'] += 1
    elif i >= 600000 and i < 800000:
        D['600000-800000'] += 1
    elif i >= 800000 and i < 1000000:
        D['800000-1000000'] += 1
    elif i >= 1000000 and i < 1200000:
        D['1000000-1200000'] += 1
    elif i >= 1200000 and i < 1400000:
        D['1200000-1400000'] += 1
    elif i >= 1400000 and i < 1600000:
        D['1400000-1600000'] += 1
    elif i >= 1600000:
        D['1600000+'] += 1

df = pd.DataFrame.from_dict(D, orient='index')
df.index.name = 'cena domu z PLN'
df.columns=['Ilość ofert']
fig = px.line(df, x=df.index, y='Ilość ofert', text='Ilość ofert')
fig.update_traces(textposition='top right')
fig.update_layout(showlegend=False)

fig.show()




In [42]:

df = data.groupby('Dzielnica').mean().reset_index()

df = df.sort_values(by=['Cena'])
fig = px.bar(df, x='Dzielnica', y='Cena', text='Cena', color='Cena')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(showlegend=False,)

fig.show()
